In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/mnt/lareaulab/reliscu/code")

from parse_gtf import *

In [3]:
psi = pd.read_csv(f"data/hahn_2023_cortex_STAR_exon_PSI.csv", index_col=0)
top_qval_mods_df = pd.read_csv("data/enrichments/hahn_2023_cortex_STAR_TPM_OR_QN_mergeParam0.85_top_MO_Qval_modules.csv")

### Add gene names to PSI data

In [4]:
# Parse GTF attribute column
gtf_file = "/mnt/lareaulab/reliscu/data/GENCODE/GRCm39/gencode.vM35.annotation.gtf"
gtf = gtf_parse(gtf_file)
gtf_subset = gtf.loc[gtf['feature'].isin(["gene"])]
attrs = gtf_subset["attribute"].apply(extract_attributes)
attrs_df = attrs.apply(pd.Series)
gtf_parsed = pd.concat([gtf_subset.drop(columns=["attribute"]), attrs_df], axis=1)

In [5]:
# Get PSI and GTF data ready to merge on gene IDs
gtf_parsed['gene_id'] = gtf_parsed['gene_id'].str.split(".").str[0]
psi['gene_id'] = psi.index.str.split("_").str[0]
psi['exon_id'] = psi.index.values

In [6]:
psi_anno = pd.merge(gtf_parsed[['gene_id', 'gene_name']], psi, on="gene_id", how="right")
psi_anno = psi_anno.set_index("exon_id").rename_axis(None)
psi_anno = psi_anno.drop(columns=["gene_id"])

### Calc. corr between ME and exon PSI

In [7]:
corr_df = pd.DataFrame(
    columns=["Gene"] + top_qval_mods_df['Cell_type'].tolist(), 
    index=psi_anno.index
)
corr_df['Gene'] = psi_anno['gene_name'] 

for i, row in top_qval_mods_df.iterrows():
    ctype = row['Cell_type']

    mod_df = pd.read_csv(row['ME_path'])
    mod_eig = mod_df.set_index("Sample")[row['Module']]
    mod_eig = pd.to_numeric(mod_eig, errors="coerce")
    
    corrs = psi_anno.iloc[:, 1:].corrwith(mod_eig, axis=1)
    corr_df[ctype] = corrs

/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/numpy/lib/

In [8]:
corr_df.head()

,Gene,YANG_PFC_2021_OLIGODENDROCYTE,BARRES_OLIGODENDROCYTES,YANG_PFC_2021_NRGN_NEURON,BAKKEN_2019_PVALB_GABAERGIC_DE_GABA_CLUSTERS,YANG_PFC_2021_L2_3_EXCITATORY_NEURON,BAKKEN_2019_SST_CHODL_GABAERGIC_DE_GABA_CLUSTERS,YANG_PFC_2021_ASTROCYTE,Mukamel_ExcitatoryNeurons,YANG_PFC_2021_OPC,...,SUGINO_UP_LAYERS4-6_GABAERGIC_NEURONS_CINGULATE_CTX,SCHIRMER_2019_OPC,Mukamel_InhibitoryNeurons,ZEISEL_MURAL,SCHIRMER_2019_MICROGLIA,SCHIRMER_2019_IN_VIP,SCHIRMER_2019_IN_PVALB,SCHIRMER_2019_IN_SV2C,SCHIRMER_2019_IN_SST,SCHIRMER_2019_EX_PYR
ENSMUSG00000025902_ProteinCoding_1,Sox17,0.092357,0.095700,0.131528,0.130839,0.124370,-0.251925,0.065036,0.128046,-0.101613,...,0.138981,-0.211470,-0.009544,0.082746,0.131194,0.096587,-0.150152,-0.170045,0.213244,-0.130622
ENSMUSG00000025902_ProteinCoding_2,Sox17,0.079432,0.083039,0.127964,0.128391,0.122055,-0.260303,0.069982,0.125595,-0.101295,...,0.137899,-0.196701,-0.000914,0.085217,0.128936,0.083223,-0.140479,-0.167023,0.206923,-0.142831
ENSMUSG00000025902_ProteinCoding_3,Sox17,-0.043981,-0.042342,0.121537,0.119390,0.119838,-0.122498,-0.096126,0.115919,-0.107698,...,0.133972,-0.094439,0.162286,-0.101081,0.119646,-0.046986,-0.094943,-0.072828,0.168193,-0.151309
ENSMUSG00000025902_ProteinCoding_4,Sox17,-0.023505,-0.023307,0.085887,0.086728,0.086286,-0.091748,-0.121977,0.084180,-0.064705,...,0.102204,-0.017887,0.109462,-0.129700,0.087563,-0.025678,-0.137717,-0.127460,0.090053,-0.097208
ENSMUSG00000033845_ProteinCoding_1,Mrpl15,-0.046297,-0.049318,-0.222226,-0.225703,-0.226101,0.137570,0.142890,-0.227870,0.220931,...,-0.220301,0.126290,-0.203138,0.186141,-0.226582,-0.044729,-0.024642,-0.136517,-0.136999,0.202147


In [10]:
corr_df.to_csv(f"data/corrs/hahn_2023_cortex_STAR_TPM_OR_QN_mergeParam0.85_top_MO_Qval_modules_exon_corr.csv")